In [ ]:
#!/usr/bin/env python3

import io

from PIL import Image
from inky.inky_uc8159 import Inky, DESATURATED_PALETTE
from matplotlib import pyplot
import seaborn
import argparse

print("""
Requires the seaborn library: sudo python3 -m pip install seaborn
You may need to: sudo apt install libatlas-base-dev
""")

# Convert the built-in palette to a list of colours usable by seaborn,
# This nets us 6 colours: Green, Blue, Red, Yellow, Orange, Black
palette_colors = [(c[0] / 255.0, c[1] / 255.0, c[2] / 255.0) for c in DESATURATED_PALETTE[2:6] + [(0, 0, 0)]]

parser = argparse.ArgumentParser()
parser.add_argument("--dataset", "-d", choices=["penguins", "dots", "mpg"], default="mpg")
args = parser.parse_args()

inky = Inky()
saturation = 0
dpi = 80
buf = io.BytesIO()

seaborn.set_theme(style="white")

if args.dataset == "mpg":
    palette = seaborn.color_palette(palette_colors, n_colors=3)
    mpg = seaborn.load_dataset("mpg")

    plot = seaborn.relplot(
        x="horsepower", y="mpg", hue="origin", size="weight",
        sizes=(40, 400), alpha=1.0, palette=palette,
        data=mpg)

if args.dataset == "penguins":
    palette = seaborn.color_palette(palette_colors, n_colors=3)
    df = seaborn.load_dataset("penguins")
    plot = seaborn.pairplot(df, hue="species", palette=palette)

if args.dataset == "dots":
    palette = seaborn.color_palette(palette_colors, n_colors=6)
    dots = seaborn.load_dataset("dots")
    plot = seaborn.relplot(
        data=dots,
        x="time", y="firing_rate",
        hue="coherence", size="choice", col="align",
        kind="line", size_order=["T1", "T2"], palette=palette,
        facet_kws=dict(sharex=False),
    )

# Force the output plot to be approximately the right size for Inky
pyplot.gcf().set_size_inches(inky.width / dpi, inky.height / dpi)

plot.savefig(buf, format="png", dpi=dpi)
buf.seek(0)

plot_image = Image.open(buf).convert("RGB")
image = Image.new("RGB", (inky.width, inky.height), (255, 255, 255))
image.paste(plot_image, (0, 0))

image.save('InkyTest.png')

#inky.set_image(image, saturation=saturation)
#inky.show()

In [1]:
#!/usr/bin/env python3

from inky.auto import auto

# To simulate:
# from inky.mock import InkyMockImpression as Inky

inky = auto(ask_user=True, verbose=True)

for y in range(inky.height - 1):
    color = y // (inky.height // 7)
    for x in range(inky.width - 1):
        inky.set_pixel(x, y, color)

inky.show()
# To simulate:
# inky.wait_for_window_close()

Detected 7-Colour (UC8159)


In [ ]:
#!/usr/bin/env python3

from inky.inky_uc8159 import Inky
import argparse
import pathlib
import struct
import sys

parser = argparse.ArgumentParser()

parser.add_argument('--type', '-t', choices=['css', 'act', 'raw', 'pal', 'gpl'], help='Type of palette to output')
parser.add_argument('--saturation', '-s', type=float, default=0.5, help='Colour palette saturation')
parser.add_argument('--file', '-f', type=pathlib.Path, help='Output file')

args = parser.parse_args()

inky = Inky()

names = ['black', 'white', 'green', 'blue', 'red', 'yellow', 'orange']

if args.file is None:
    print("You must specify an output palette file.")
    sys.exit(1)


def raw_palette():
    palette = bytearray(768)
    palette[0:8 * 3] = inky._palette_blend(args.saturation, dtype='uint8')
    return palette


if args.type == 'css':
    palette = inky._palette_blend(args.saturation, dtype='uint24')
    with open(args.file, 'w+') as f:
        for i in range(7):
            name = names[i]
            colour = palette[i]
            f.write(""".{name}_fg {{font-color:#{colour:06x}}}
.{name}_bg {{background-color:#{colour:06x}}}
""".format(name=name, colour=colour))

if args.type == 'gpl':
    palette = inky._palette_blend(args.saturation, dtype='uint24')
    with open(args.file, 'w+') as f:
        f.write("GIMP Palette\n")
        f.write("Name: InkyImpressions\n")
        f.write("Columns: 7\n")
        for i in range(7):
            name = names[i]
            colour = palette[i]
            r = (colour & 0xff0000) >> 16
            g = (colour & 0x00ff00) >> 8
            b = (colour & 0x0000ff)
            f.write("{r} {g} {b} Index {i} # {name}\n".format(r=r, g=g, b=b, i=i, name=name))

if args.type in ('pal', 'raw'):
    palette = raw_palette()
    with open(args.file, 'wb+') as f:
        f.write(palette)

if args.type == 'act':
    palette = raw_palette()
    palette += struct.pack(">HH", 7, 0xFFFF)
    with open(args.file, 'wb+') as f:
        f.write(palette)